In [ ]:
import pathlib as path
import sys
import docker as docker
import json as json
import pandas as pd
import aiohttp
import asyncio

In [ ]:
# Establish notebook path for handling relative paths in the notebook
notebook_path = path.Path().resolve()

if notebook_path.stem != "dev":
    raise Exception(
        "Notebook file root must be set to parent directory of the notebook. Please resolve and re-run."
    )

# Add the thesis directory to the path for importing local modules
sys.path.append(str(notebook_path.parent.parent))

import pharmalink.code.area as area
import pharmalink.code.sources as src
import pharmalink.code.customers as cust
import pharmalink.code.valhalla as valhalla

In [ ]:
# regkey = "09663"
regkey = "09162"

model_area = area.Area(regkey)
pharmacies = src.Pharmacies.get_within_area(model_area)
customers = cust.Customers(model_area).customers

In [ ]:
valhalla_instance = valhalla.ValhallaInstance().valhalla

In [ ]:
# Goal is to select one of the 3 closest pharmacies for each customer with a probability based on distance
ph_locs = pharmacies.to_crs(epsg=25832).geometry
cus_locs = customers.to_crs(epsg=25832).geometry

chosen_pharmacies = []

for cus in cus_locs:
    # Find the 3 pharmacies closest to the customer
    closest = ph_locs.distance(cus).sort_values(axis=0, ascending=True).head(3)

    # Choose from the nearest pharmacies with a probability based on distance
    chosen_pharmacies.append(closest.sample(weights=1 / closest).index[0])

customers["chosen_pharmacy"] = chosen_pharmacies

In [ ]:
async def fetch_trip(session, cus_id, locations, mot):
    params = {
        "id": cus_id,
        "locations": locations,
        "costing": mot,
        "units": "kilometers",
        "directions_type": "none",
    }

    # print(f"Fetching trip for customer {cus_id} with {mot}...")

    async with session.get("/route", json=params) as response:
        return mot, await response.json()


async def process_customer(session, cus):
    cus_id = cus[0]
    cus_loc = cus[1]
    ph_loc = pharmacies.loc[cus[2], "geometry"]

    start = end = {"lat": cus_loc.y, "lon": cus_loc.x}
    target = {"lat": ph_loc.y, "lon": ph_loc.x}

    locations = [start, target, end]

    tasks = []
    for mot in ["auto", "bicycle", "pedestrian"]:
        tasks.append(fetch_trip(session, cus_id, locations, mot))

    # results = {cus[0]: {}}

    results = {}

    for task in asyncio.as_completed(tasks):
        mot, result = await task

        # Check if response id matches customer id
        # if result["id"] == cus_id:
        #    del result["id"]

        if "trip" in result:
            results[mot] = result["trip"]

    avg_length = sum([results[mot]["summary"]["length"] for mot in results]) / len(
        results
    )

    # Choose a probable mode of transport based on the average trip length
    chosen_mot = src.evaluate_mode_of_transport(avg_length, results.keys())

    # Add the chosen mode of transport to the results
    results[chosen_mot]["mot"] = chosen_mot

    return {cus_id: results[chosen_mot]}


async def calculate_trips():

    # Start Valhalla container
    valhalla_instance.start()

    # Wait until Valhalla is ready
    while valhalla_instance.health != "healthy":
        await asyncio.sleep(0.1)
        valhalla_instance.reload()

    await asyncio.sleep(1)

    # Find Valhalla API endpoint
    valhalla_api = f"http://localhost:8002"

    trips = {}

    async with aiohttp.ClientSession(
        base_url=valhalla_api,
        timeout=aiohttp.ClientTimeout(600),
        connector=aiohttp.TCPConnector(limit=100),
    ) as session:

        tasks = []
        for cus in customers.itertuples():
            tasks.append(process_customer(session, cus))

        for task in asyncio.as_completed(tasks):
            trips.update(await task)

    # Stop Valhalla container
    valhalla_instance.stop()

    return trips

In [ ]:
%autoawait

# Run the main function
raw_trips = await calculate_trips()

In [ ]:
trips = pd.DataFrame.from_dict(
    raw_trips, orient="index", columns=["locations", "legs", "summary", "mot"]
)
trips.index.name = "cus_id"
trips.index = trips.index.astype(int)
trips = trips.sort_index()
trips = trips.join(pd.json_normalize(trips["summary"]))
trips = trips.drop(
    columns=["summary", "has_time_restrictions", "has_toll", "has_highway", "has_ferry"]
)

In [ ]:
total = trips.groupby("mot").sum()
total = total[["time", "length"]]
total["time"] = total["time"] / 3600
total